In [10]:
import torch
import torch.nn.functional as F 
from torch_geometric.datasets import WikiCS,  CitationFull,  Coauthor, KarateClub
from torch_geometric.data import Data
import utils
from utils import count_out_degree, make_adj_matrix, count_in_degree, create_pointer_graph
import math
import copy
import random
import matplotlib.pyplot

In [67]:

edges0 = torch.tensor([[0, 0, 0], [1, 2, 3]])
edges1 = torch.cat((edges0, edges0+1, edges0+2, edges0+3), dim=1)%4
edges2 = torch.cat((edges1, edges1+4, edges1+8, edges1+12), dim=1)
l = [i*2+2 for i in range(8)]
l2= [[i*2+2, i*2+4] for i in range(0, 8, 2)]
l2 = [nek for nes in l2 for nek in nes[::-1]]
edges_extra = torch.tensor([l, l2])%16
edges3 = torch.cat((edges2, edges_extra), dim=1)
x = torch.zeros(16, 1)
graph = Data(edge_index=edges3, x = x)

edgesextra = torch.tensor([[0, 15, 16, 31, 32, 47, 48, 63], [63, 16, 15, 32, 31, 48, 47, 0]])
edges64 = torch.cat((edges3, edges3+16, edges3+32, edges3+32+16, edgesextra), dim=1)
graph64 = Data(edge_index=edges64, x= torch.zeros(64, 1))
graph64

Data(edge_index=[2, 232], x=[64, 1])

In [58]:
graph2 = KarateClub().data
graph2.num_nodes = 34

In [70]:
def louvain_communities(graph):
    hierarchy = []
    graph = copy.deepcopy(graph)
    hpg = create_weighted_hashed_pointer_graph(graph)
    deg = count_out_degree(graph, aslist=True) 
    communities = [[{i}, 0, d] for i, d in enumerate(deg)]
    node2comm = [i for i in range(graph.num_nodes)]
    div = len(graph.edge_index[0])

    def get_degin(node, comm):
        degin = 0
        for candidate in hpg[node][0]:
            if candidate in comm[0]:
                degin += hpg[node][0][candidate]
        return degin

    def collapse_communities(communities):
        hpg2 = []
        for i, comm in enumerate(communities):
            tmp = dict()
            leni = 0
            for node in comm[0]:
                # print(node, "node")
                try: hpg[node][0]
                except: print(hpg, "coommmmmmssm", communities)
                for node2 in hpg[node][0]:
                    if node2comm[node2] in tmp:
                        tmp[node2comm[node2]] += hpg[node][0][node2]
                    else:  
                        tmp[node2comm[node2]] = hpg[node][0][node2]
                    # leni += hpg[node][0][node2]
            # comm[]
            tmp[i] //= 2
            # print(i, comm, leni)
            hpg2.append([tmp, 0])
            deg[i] = comm[1]+comm[2]
        for i, comm in enumerate(communities):
            comm[0] = {i}

        graph.num_nodes = len(communities)
        return hpg2
                

    while len(communities)>1:
        for ups in range(20):
            made_switch = False
            for node in range(graph.num_nodes):
                print(node, end='\r')
                max_deltaQ, maxi = float('-inf'), -1
                maxdegin = None
                selfcomm, iselfcomm, selfdegin, selfdelta = None, None, None, None
                for i, comm in enumerate(communities):
                    degin = get_degin(node, comm)
                    frst = (comm[1]+degin)/div - ((comm[2]+deg[node])/div)**2
                    scnd = comm[1]/div - (comm[2]/div)**2 - (degin/div)**2
                    deltaQ = frst - scnd
                    if node in comm[0]:
                        selfcomm, iselfcomm, selfdegin, selfdelta = comm, i, degin, deltaQ
                        continue
                    if deltaQ > max_deltaQ:
                        max_deltaQ, maxcomm, maxdegin, maxi = deltaQ, comm, degin, i
                if max_deltaQ-selfdelta > 1e-10:
                    made_switch = True
                    selfcomm[0].remove(node)
                    selfcomm[1] -= selfdegin
                    selfcomm[2] -= deg[node] - 2*selfdegin
                    maxcomm[0].add(node)
                    maxcomm[1] += maxdegin
                    maxcomm[2] += deg[node] - 2*maxdegin
                if len(selfcomm[0]) == 0:
                    del communities[iselfcomm] #expendive ???
            for i, comm in enumerate(communities):
                for node in comm[0]:
                    node2comm[node] = i
            if not made_switch:
                hierarchy.append(copy.deepcopy(communities))
                print("break")
                break
        hpg = collapse_communities(communities)
    return hierarchy

hier = louvain_communities(graph64) 
hier

break
break
break
break
break
break
break
break
break


[[[{0, 1, 2, 3}, 6, 3],
  [{4, 5, 6, 7}, 6, 2],
  [{8, 9, 10, 11}, 6, 2],
  [{12, 13, 14, 15}, 6, 3],
  [{16, 17, 18, 19}, 6, 3],
  [{20, 21, 22, 23}, 6, 2],
  [{24, 25, 26, 27}, 6, 2],
  [{28, 29, 30, 31}, 6, 3],
  [{32, 33, 34, 35}, 6, 3],
  [{36, 37, 38, 39}, 6, 2],
  [{40, 41, 42, 43}, 6, 2],
  [{44, 45, 46, 47}, 6, 3],
  [{48, 49, 50, 51}, 6, 3],
  [{52, 53, 54, 55}, 6, 2],
  [{56, 57, 58, 59}, 6, 2],
  [{60, 61, 62, 63}, 6, 3]],
 [[{0}, 6, 3],
  [{1}, 6, 2],
  [{2}, 6, 2],
  [{3}, 6, 3],
  [{4}, 6, 3],
  [{5}, 6, 2],
  [{6}, 6, 2],
  [{7}, 6, 3],
  [{8}, 6, 3],
  [{9}, 6, 2],
  [{10}, 6, 2],
  [{11}, 6, 3],
  [{12}, 6, 3],
  [{13}, 6, 2],
  [{14}, 6, 2],
  [{15}, 6, 3]],
 [[{0}, 6, 3],
  [{1}, 6, 2],
  [{2}, 6, 2],
  [{3}, 6, 3],
  [{4}, 6, 3],
  [{5}, 6, 2],
  [{6}, 6, 2],
  [{7}, 6, 3],
  [{8}, 6, 3],
  [{9}, 6, 2],
  [{10}, 6, 2],
  [{11}, 6, 3],
  [{12}, 6, 3],
  [{13}, 6, 2],
  [{14}, 6, 2],
  [{15}, 6, 3]],
 [[{0, 1}, 7, 9],
  [{2, 3}, 7, 9],
  [{4, 5}, 7, 9],
  [{6, 7}, 7,

In [43]:
def create_hashed_pointer_graph(graph):
    hashed_pointer_graph = create_pointer_graph(graph)
    hpg = hashed_pointer_graph
    for i in range(len(hpg)):
        l1, l2 = hpg[i]
        h1 = { i for i in l1}
        h2 = { i for i in l2}
        hpg[i] = [h1, h2]
    return hpg
def create_weighted_hashed_pointer_graph(graph):
    hashed_pointer_graph = create_pointer_graph(graph)
    hpg = hashed_pointer_graph
    for i in range(len(hpg)):
        l1, l2 = hpg[i]
        h1 = { i:1 for i in l1}
        h2 = { i:1 for i in l2}
        hpg[i] = [h1, h2]
    return hpg

# create_hashed_pointer_graph(graph2)
create_weighted_hashed_pointer_graph(graph)

[[{1: 1, 2: 1, 3: 1, 14: 1}, {1: 1, 2: 1, 3: 1, 14: 1}],
 [{2: 1, 3: 1, 0: 1}, {0: 1, 2: 1, 3: 1}],
 [{3: 1, 0: 1, 1: 1, 4: 1}, {0: 1, 1: 1, 3: 1, 4: 1}],
 [{0: 1, 1: 1, 2: 1}, {0: 1, 1: 1, 2: 1}],
 [{5: 1, 6: 1, 7: 1, 2: 1}, {5: 1, 6: 1, 7: 1, 2: 1}],
 [{6: 1, 7: 1, 4: 1}, {4: 1, 6: 1, 7: 1}],
 [{7: 1, 4: 1, 5: 1, 8: 1}, {4: 1, 5: 1, 7: 1, 8: 1}],
 [{4: 1, 5: 1, 6: 1}, {4: 1, 5: 1, 6: 1}],
 [{9: 1, 10: 1, 11: 1, 6: 1}, {9: 1, 10: 1, 11: 1, 6: 1}],
 [{10: 1, 11: 1, 8: 1}, {8: 1, 10: 1, 11: 1}],
 [{11: 1, 8: 1, 9: 1, 12: 1}, {8: 1, 9: 1, 11: 1, 12: 1}],
 [{8: 1, 9: 1, 10: 1}, {8: 1, 9: 1, 10: 1}],
 [{13: 1, 14: 1, 15: 1, 10: 1}, {13: 1, 14: 1, 15: 1, 10: 1}],
 [{14: 1, 15: 1, 12: 1}, {12: 1, 14: 1, 15: 1}],
 [{15: 1, 12: 1, 13: 1, 0: 1}, {12: 1, 13: 1, 15: 1, 0: 1}],
 [{12: 1, 13: 1, 14: 1}, {12: 1, 13: 1, 14: 1}]]